In [ ]:
!pip install yfinance gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
from re import VERBOSE
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

def predict_stock_prices(stock_symbol, start_date, end_date, future_days=30, confidence_margin=0.02):
    try:
        # Fetch stock data
        data = yf.download(stock_symbol, start=start_date, end=end_date)
        if data.empty:
            return "No data available for the given stock symbol and date range."

        # Preprocess data
        close_prices = data['Close'].values.reshape(-1, 1)
        scaler = MinMaxScaler(feature_range=(0, 1))
        close_prices_scaled = scaler.fit_transform(close_prices)

        # Create LSTM data
        def create_lstm_data(data, time_steps=1):
            x, y = [], []
            for i in range(len(data) - time_steps):
                x.append(data[i:(i + time_steps), 0])
                y.append(data[i + time_steps, 0])
            return np.array(x), np.array(y)

        time_steps = 10
        x, y = create_lstm_data(close_prices_scaled, time_steps)
        x = np.reshape(x, (x.shape[0], x.shape[1], 1))

        # Build LSTM model
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(x.shape[1], 1)))
        model.add(LSTM(units=50))
        model.add(Dense(units=1))
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Train the model
        model.fit(x, y, epochs=10, batch_size=32, verbose=0)

        # Predict future prices iteratively
        last_prices = close_prices[-time_steps:]
        last_prices_scaled = scaler.transform(last_prices.reshape(-1, 1))

        future_predictions = []
        input_sequence = last_prices_scaled.copy()

        for _ in range(future_days):
            x_pred = np.reshape(input_sequence, (1, time_steps, 1))
            predicted_scaled = model.predict(x_pred, verbose=0)
            future_predictions.append(predicted_scaled[0, 0])
            input_sequence = np.append(input_sequence[1:], predicted_scaled, axis=0)

        # Scale predictions back to original values
        future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

        # Calculate range values (±2% margin)
        lower_bound = future_predictions * (1 - confidence_margin)
        upper_bound = future_predictions * (1 + confidence_margin)

        # Prepare future dates and predictions
        future_dates = pd.date_range(start=pd.to_datetime(end_date) + pd.Timedelta(days=1), periods=future_days)
        future_data = pd.DataFrame({
            'Date': future_dates,
            'Predicted Price': future_predictions.flatten(),
            'Lower Bound': lower_bound.flatten(),
            'Upper Bound': upper_bound.flatten()
        })

        return future_data

    except Exception as e:
        return str(e)


In [17]:
#n8n Integration
import requests

# Replace with your actual n8n webhook URL
N8N_WEBHOOK_URL = "n8n link"
def send_to_n8n(data):
    """Sends time series data to the n8n webhook and retrieves response."""
    try:
        response = requests.post(N8N_WEBHOOK_URL, json={"data": data})
        return response.json().get("reply", "No response from n8n")
    except Exception as e:
        return f"Error: {e}"

    # chatbot_input = gr.Textbox(label="Enter your message")
    # chatbot_output = gr.Textbox(label="n8n Response")


In [18]:
# Gradio interface

def gradio_interface(stock_symbol, start_date, end_date, future_days=30, confidence_margin=0.02):

    prediction = predict_stock_prices(stock_symbol, start_date, end_date, future_days,confidence_margin)

    return prediction



# Set up Gradio UI

inputs = [

    gr.Textbox(label="Stock Symbol", placeholder="e.g., AAPL"),

    gr.Textbox(label="Start Date", placeholder="YYYY-MM-DD"),

    gr.Textbox(label="End Date", placeholder="YYYY-MM-DD"),

    gr.Slider(1, 60, value=30, step=1, label="Number of Future Days to Predict")

]

In [ ]:
outputs = gr.Dataframe(label="Predicted Stock Prices")

In [ ]:
gr.Interface(fn=gradio_interface, inputs=inputs, outputs=outputs, title="Stock Price Prediction").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6cd3bbd4b7d2da18e2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
